<a href="https://colab.research.google.com/github/pratikunterwegs/holeybirds/blob/master/get_elev_indices_hula.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!/usr/bin/env python
# coding: utf-8

import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as geemap
except:
    import geemap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

geemap package not installed. Installing ...
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=RbH1Usb0yO2nSyB7FEqKP--a4DFbrlbBXuWiTtrKUd0&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhqW15U7bxkD0ERp4g5_JBfsxD8WKsf7vq62lFu5VXCba-yxO-UJr4

Successfully saved authorization token.


In [14]:
# get study area bounding box
hula = ee.FeatureCollection("users/pratik_unterwegs/hula_valley");

# get elevation data
dataset = ee.Image('USGS/SRTMGL1_003');
elevation = dataset.select('elevation');

# hula valley 10km buffer
hula_buffer = hula.geometry()
hula_buffer = hula_buffer.buffer(10000)

# define start and end date june and dec 2016
start_date = '2016-06-01'
end_date = '2016-12-01'

# clip elevation data
elevation_clip = elevation.clip(hula_buffer);

# get sentinel data for the time range 2016
sentinel = ee.ImageCollection("COPERNICUS/S2")
sentinel_bounds = sentinel.filterBounds(hula_buffer)
sentinel_2016 = sentinel_bounds.filterDate(start_date, end_date)

In [3]:

# adding a NDVI band
def add_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('ndvi')
    return image.addBands([ndvi])


# add an NDBI band
def add_ndbi(image):
    ndbi = image.normalizedDifference(['B11', 'B8']).rename('ndbi')
    return image.addBands([ndbi])

In [15]:
# map metrics over sentinel
sentinel_ndvi = sentinel_2016.map(add_ndvi)
sentinel_ndbi = sentinel_2016.map(add_ndbi)

# get median values
ndvi_median = sentinel_ndvi.select('ndvi').median()
ndbi_median = sentinel_ndbi.select('ndbi').median()

# clip to geometry
ndvi_clip = ndvi_median.clip(hula_buffer)
ndbi_clip = ndbi_median.clip(hula_buffer)

# get built up prob as ndbi - ndvi
built = ndbi_median.subtract(ndvi_median)
built_clip = built.clip(hula_buffer)

In [17]:
# Define an sld style color ramp to apply to the image.
sld_ramp = \
  '<RasterSymbolizer>' + \
    '<ColorMap type="ramp" extended="false" >' + \
      '<ColorMapEntry color="#0D0887" quantity="-1" label="0"/>' + \
      '<ColorMapEntry color="#CC4678" quantity="0" label="50" />' + \
      '<ColorMapEntry color="#F0F921" quantity="0.5" label="100" />' + \
    '</ColorMap>' + \
  '</RasterSymbolizer>';


# print layers
vis = {'bands': ['elevation']}
Map = geemap.Map(center=[33.0005,33.5], zoom=9)
Map.addLayer(ndvi_clip.sldStyle(sld_ramp), {}, 'ndvi')
Map.addLayer(hula)
Map.addLayerControl()

Map

In [18]:
# export image to drive
downConfig = {'scale': 30, "maxPixels": 1.0E13, 'driveFolder': 'hula_rasters'}  # scale means resolution.
name = "raster_hula_srtm_30"
# print(name)
task = ee.batch.Export.image(elevation_clip, name, downConfig)
task.start()

In [19]:
# export image to drive
downConfig = {'scale': 10, "maxPixels": 1.0E13, 'driveFolder': 'hula_rasters'}  # changed scale for sentinel
name = "raster_hula_builtup"
# print(name)
task = ee.batch.Export.image(built_clip, name, downConfig)
task.start()

# export ndvi to drive
name_ndvi = "raster_hula_ndvi"
# print(name)
task_ndvi = ee.batch.Export.image(ndvi_clip, name_ndvi, downConfig)
task_ndvi.start()